# Meteo con el pato

### Este informe muestra los datos diarios de temperatura recogidos en el portal de datos abiertos del ayuntamiento.
* dataset: **meteorologia-tiempo-real** 

In [22]:
import duckdb
import plotly.express as px

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%%capture

#conexion a bbdd en memoria o en fichero
#conn = duckdb.connect() 
conn = duckdb.connect('/home/jcaubin/duck_test.db') 


%sql conn --alias duckdb
#limite de filas a mostrar
%config SqlMagic.displaylimit = 100 

In [24]:
# imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f"### hora de ejecución: {t}"))

### hora de ejecución: 2025-01-25 11:27:47.895860

## METEO

In [25]:

magnitud = 83 #temperatura

In [26]:
%%sql 
result2 << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = {{magnitud}} 
and validez = 'V'
and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h


Running query in 'duckdb'

In [27]:
df = result2.DataFrame()

In [28]:
import plotly.express as px

#df = result2.DataFrame()

fig = px.line(df, x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='Temperaturas hoy')
fig.show()

In [29]:
%%sql
--tabla ancha
PIVOT(
select estacion_desc, h, valor
from CALAIR
where
magnitud = {{magnitud}} 
and validez = 'V'
and DIA = datepart('day',current_date())
)
on ESTACION_DESC
USING sum(VALOR)
group by H

Running query in 'duckdb'

H,Arturo Soria,Barrio del Pilar,Casa de Campo,Centro Mpal. De Acústica,Cuatro Caminos,El Pardo,Ensanche de Vallecas,Escuelas Aguirre,Farolillo,J.M.D. Hortaleza,J.M.D. Moratalaz,J.M.D.Centro,J.M.D.Chamartin,J.M.D.Chamberí,J.M.D.Vallecas 1,J.M.D.Vallecas 2,Juan Carlos I,Matadero 02,Moratalaz,Peñagrande,Plaza Elíptica,Plaza España,Plaza del Carmen
1,11.4,10.9,10.7,9.0,10.7,10.2,10.7,11.2,12.0,8.4,7.3,11.3,11.0,11.3,11.1,12.0,10.7,11.7,10.7,8.1,11.6,11.4,11.4
2,11.0,10.7,10.3,8.8,10.3,10.2,10.3,10.8,11.5,7.9,7.0,10.8,10.6,10.9,10.8,11.8,10.4,11.4,10.4,7.7,11.1,10.9,10.9
3,10.6,10.5,10.0,8.6,9.9,10.1,10.4,10.4,11.1,7.7,6.9,10.5,10.3,10.6,10.6,11.6,10.2,11.1,10.2,7.5,10.7,10.6,10.5
4,10.4,10.1,9.7,8.2,9.6,9.8,10.1,10.3,10.8,7.5,6.5,10.2,10.1,10.4,10.1,11.3,9.9,10.5,9.8,7.2,10.1,10.5,10.3
5,10.2,9.8,9.6,8.1,9.5,9.4,9.6,10.0,10.5,7.4,6.0,9.9,9.9,10.3,9.8,11.0,9.6,10.2,9.6,7.2,10.0,10.3,10.1
6,9.8,9.8,9.5,8.0,9.3,9.5,9.2,9.6,10.3,7.0,5.8,9.7,9.6,9.9,9.6,10.7,9.3,10.2,9.2,7.2,9.9,9.9,9.8
7,9.6,9.5,9.2,7.8,9.1,9.4,9.1,9.4,9.9,None,5.6,9.5,9.4,9.7,9.5,10.6,9.0,9.9,9.0,None,9.5,9.7,9.5
8,9.4,9.2,8.7,7.4,8.8,9.2,8.5,9.2,9.7,None,5.3,9.3,9.2,9.5,9.2,10.4,9.0,9.6,8.7,None,9.2,9.4,9.3
9,9.2,9.1,8.7,7.2,8.7,8.8,8.5,9.1,9.8,None,5.2,9.3,9.1,9.5,9.1,10.3,8.6,9.6,8.7,None,9.3,9.5,9.3
10,9.0,8.8,8.4,7.0,8.3,8.4,8.6,9.0,9.7,None,None,9.1,8.8,9.3,8.9,10.3,8.5,None,8.4,None,9.2,9.2,9.1


In [30]:
%%sql 
precipitaciones << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = 89
and validez = 'V'
and estacion_desc in ['Casa de Campo'] --, 'Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h


Running query in 'duckdb'

In [31]:
fig = px.line(precipitaciones.DataFrame(), x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='Precipitaciones ')
fig.show()

## CALIDAD DEL AIRE

In [32]:
%%sql 
nox << 
select estacion_desc,dia,  h, valor 
from CALAIR 
where magnitud = 08
and validez = 'V'
--and estacion_desc in ['Casa de Campo','Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h

Running query in 'duckdb'

In [33]:
fig = px.line(nox.DataFrame(), x="H", y="VALOR", color="DIA", symbol='ESTACION_DESC', title='NO2 ')
fig.show()

In [34]:
fig = px.line(nox.DataFrame(), x="H", y="VALOR", color="DIA", facet_col='ESTACION_DESC',facet_col_wrap=4 ,title='NOX ')
fig.show()

In [35]:
%%sql
--tabla ancha
PIVOT(
select estacion_desc, h, valor
from calair
where
magnitud = 08
and validez = 'V'
and DIA = datepart('day',current_date())
)
on ESTACION_DESC
USING sum(VALOR)
group by H

Running query in 'duckdb'

H,Arturo Soria,Barrio del Pilar,Casa de Campo,Cuatro Caminos,El Pardo,Ensanche de Vallecas,Escuelas Aguirre,Farolillo,Juan Carlos I,Moratalaz,Plaza Elíptica,Plaza España,Plaza del Carmen
1,17.0,15.0,16.0,12.0,9.0,23.0,27.0,15.0,24.0,25.0,18.0,17.0,15.0
2,8.0,7.0,10.0,8.0,7.0,13.0,15.0,8.0,12.0,17.0,11.0,8.0,8.0
3,6.0,5.0,5.0,5.0,4.0,14.0,12.0,5.0,7.0,10.0,6.0,6.0,7.0
4,4.0,3.0,5.0,5.0,3.0,10.0,10.0,4.0,5.0,5.0,4.0,6.0,5.0
5,5.0,3.0,5.0,4.0,2.0,7.0,9.0,4.0,5.0,7.0,4.0,6.0,4.0
6,4.0,3.0,5.0,5.0,2.0,13.0,9.0,4.0,5.0,5.0,5.0,5.0,4.0
7,5.0,5.0,6.0,7.0,2.0,10.0,10.0,6.0,6.0,7.0,6.0,7.0,5.0
8,8.0,7.0,5.0,8.0,2.0,11.0,13.0,9.0,10.0,11.0,11.0,9.0,8.0
9,10.0,11.0,7.0,11.0,3.0,17.0,18.0,16.0,13.0,18.0,15.0,16.0,11.0


In [45]:
%%sql 
cont <<
select estacion_desc, parametro,dia,  h, valor 
from CALAIR 
where DIA = datepart('day',current_date())
and validez = 'V'
and magnitud <80
--and estacion_desc in ['Casa de Campo','Juan Carlos I', 'El Pardo','Peñagrande', 'Plaza Elíptica']
order by estacion_desc, h


Running query in 'duckdb'

In [52]:
fig = px.line(cont.DataFrame(), x="H", y="VALOR", color="ESTACION_DESC", facet_col='PARAMETRO',facet_col_wrap=4 ,title='Contaminantes ')
fig.update_yaxes(matches=None)
fig.show()

In [14]:
%sql --close duckdb